### Librerias

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from joblib import dump, load
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from joblib import dump, load


### Informacion general sobre DataFrame

In [20]:
# Cargar el dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/sales.csv"
df = pd.read_csv(url)

In [21]:
# Mostrar las primeras filas y la información general del dataset
df.head()

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales
0,0,625,3,2013-11-06,641,1,1,0,0,7293
1,1,293,2,2013-07-16,877,1,1,0,1,7060
2,2,39,4,2014-01-23,561,1,1,0,0,4565
3,3,676,4,2013-09-26,1584,1,1,0,0,6380
4,4,709,3,2014-01-22,1477,1,1,0,0,11647


### Transformacion de los datos

In [22]:
# Convertir la columna 'Date' a tipo datetime 
df['Date'] = pd.to_datetime(df['Date'])

# Obtener la primera fecha 
primera_fecha = df['Date'].min()

print("La primera fecha en la columna 'Date' es:", primera_fecha)


La primera fecha en la columna 'Date' es: 2013-01-01 00:00:00


In [23]:
df = pd.DataFrame(df)

# Asegurarse de que la columna 'Date' está en formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Fecha de referencia
fecha_referencia = pd.to_datetime('2013-01-01')

# Crear la nueva columna 'date -1' con la diferencia en días
df['date -1'] = (df['Date'] - fecha_referencia).dt.days

# Mostrar el DataFrame para verificar
df.head()


,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,date -1
0,0,625,3,2013-11-06,641,1,1,0,0,7293,309
1,1,293,2,2013-07-16,877,1,1,0,1,7060,196
2,2,39,4,2014-01-23,561,1,1,0,0,4565,387
3,3,676,4,2013-09-26,1584,1,1,0,0,6380,268
4,4,709,3,2014-01-22,1477,1,1,0,0,11647,386


In [24]:
df = pd.DataFrame(df)

# Asegurarse de que 'Day_of_week' es numérico
df['Day_of_week'] = pd.to_numeric(df['Day_of_week'], errors='coerce')

# Calcular seno y coseno directamente usando Day_of_week
df['Seno'] = np.sin(2 * np.pi * df['Day_of_week'] / 7)
df['Coseno'] = np.cos(2 * np.pi * df['Day_of_week'] / 7)

# Mostrar el DataFrame resultante
df.head()

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,date -1,Seno,Coseno
0,0,625,3,2013-11-06,641,1,1,0,0,7293,309,0.433884,-0.900969
1,1,293,2,2013-07-16,877,1,1,0,1,7060,196,0.974928,-0.222521
2,2,39,4,2014-01-23,561,1,1,0,0,4565,387,-0.433884,-0.900969
3,3,676,4,2013-09-26,1584,1,1,0,0,6380,268,-0.433884,-0.900969
4,4,709,3,2014-01-22,1477,1,1,0,0,11647,386,0.433884,-0.900969


In [25]:
df.dtypes

True_index                      int64
Store_ID                        int64
Day_of_week                     int64
Date                   datetime64[ns]
Nb_customers_on_day             int64
Open                            int64
Promotion                       int64
State_holiday                  object
School_holiday                  int64
Sales                           int64
date -1                         int64
Seno                          float64
Coseno                        float64
dtype: object

In [26]:
# Crear variables dummy para State_holiday
df = pd.get_dummies(df, columns=['State_holiday'], prefix='State_holiday', drop_first=True)

# Verificar los nuevos tipos de datos
df.dtypes

True_index                      int64
Store_ID                        int64
Day_of_week                     int64
Date                   datetime64[ns]
Nb_customers_on_day             int64
Open                            int64
Promotion                       int64
School_holiday                  int64
Sales                           int64
date -1                         int64
Seno                          float64
Coseno                        float64
State_holiday_a                  bool
State_holiday_b                  bool
State_holiday_c                  bool
dtype: object

### Split y escalado 

In [27]:
# Definimos las características (X) y la etiqueta (y)
X = df[['Store_ID', 'Nb_customers_on_day', 'Open', 'Promotion', 
         'State_holiday_a', 'State_holiday_b', 'State_holiday_c', 
         'School_holiday', 'date -1', 'Seno', 'Coseno']]
y = df['Sales']  # Target

# Hacemos el split en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")


Tamaño del conjunto de entrenamiento: 512672
Tamaño del conjunto de prueba: 128168


In [28]:
# Instanciar el escalador
scaler = StandardScaler()

# Escalar solo las características numéricas
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)        


## Comprobacion de los modelos

### LinearRegression

In [29]:
# Crear el modelo
model = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento
model.fit(X_train_scaled, y_train)


LinearRegression()

In [30]:
# Hacer predicciones
y_pred = model.predict(X_test_scaled)

In [31]:
# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
# Calcular R²
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


RMSE: 1471.49
R²: 0.86


c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
# Realizar validación cruzada
scores = cross_val_score(model, X_train_scaled, y_train, cv=5)

print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean()}")


Cross-validation scores: [0.85640452 0.8551891  0.85899935 0.85666548 0.8568562 ]
Mean cross-validation score: 0.8568229309296298


In [33]:
# Guardar el modelo
joblib.dump(model, 'mi_modelo.pkl')

['mi_modelo.pkl']

### DecisionTreeRegressor

In [34]:
# Instanciar el modelo de Árbol de Decisión
model = DecisionTreeRegressor(random_state=42)

# Ajustar el modelo con los datos de entrenamiento
model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

RMSE: 1218.07
R²: 0.90


c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### RandomForestRegressor

In [35]:
# Instanciar y entrenar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# Guardar el modelo
dump(model, 'random_forest_model.joblib')

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 876.36
R²: 0.95


['random_forest_model.joblib']

### KNeighborsRegressor

In [36]:
# Instanciar y entrenar el modelo de KNN
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# Guardar el modelo
dump(model, 'knn_model.joblib')

RMSE: 1366.17
R²: 0.88


c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


['knn_model.joblib']